<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisites</a></span></li><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instructions</a></span></li><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports and Constants</a></span></li><li><span><a href="#Constants" data-toc-modified-id="Constants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constants</a></span></li><li><span><a href="#Export-Images" data-toc-modified-id="Export-Images-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Export Images</a></span></li></ul></div>

## Pre-requisites
Register a Google account at [https://code.earthengine.google.com](https://code.earthengine.google.com). This process may take a couple of days. Without registration, the `ee.Initialize()` command below will throw an error message.

## Instructions

This notebook exports Landsat satellite image composites of DHS and LSMS clusters from Google Earth Engine.

The images are saved in gzipped TFRecord format. By default, this notebook exports images to Google Drive. If you instead prefer to export images to Google Cloud Storage (GCS), change the `EXPORT` constant below to `'gcs'` and set `BUCKET` to the desired GCS bucket name.


|      | Storage  | Expected Export Time
|------|----------|---------------------
| DHS  | ~16.0 GB | ~24h
| LSMS |  ~2.5 GB | ~10h

The exported images take up a significant amount of storage space. Before exporting, make sure you have enough storage space. The images are exported to the following locations, based on the constants `EXPORT` and `BUCKET` defined below:

|      | Google Drive (default) | GCS
|------|:-----------------------|:---
| DHS  | `dhs_tfrecords_raw/`   | `{BUCKET}/dhs_tfrecords_raw/`
| LSMS | `lsms_tfrecords_raw/`  | `{BUCKET}/lsms_tfrecords_raw/`

Once the images have finished exporting, download the exported TFRecord files to the following folders:

- DHS: `data/dhs_tfrecords_raw/`
- LSMS: `data/lsms_tfrecords_raw/`

The folder structure should look as follows:

```
data/
    dhs_tfrecords_raw/
        angola_2011_00.tfrecord.gz
        ...
        zimbabwe_2015_00.tfrecord.gz
    lsms_tfrecords_raw/
        ethiopia_2011_00.tfrecord.gz
        ...
        uganda_2013_00.tfrecord.gz
```

## Imports and Constants

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import math
from typing import Any, Dict, Optional, Tuple

import ee
import pandas as pd

import ee_utils

/home/saori/.pyenv/versions/3.10.6/envs/poverty_mapper/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Before using the Earth Engine API, you must perform a one-time authentication that authorizes access to Earth Engine on behalf of your Google account you registered at [https://code.earthengine.google.com](https://code.earthengine.google.com). The authentication process saves a credentials file to `$HOME/.config/earthengine/credentials` for future use.

The following command `ee.Authenticate()` runs the authentication process. Once you successfully authenticate, you may comment out this command because you should not need to authenticate again in the future, unless you delete the credentials file. If you do not authenticate, the subsequent `ee.Initialize()` command below will fail.

For more information, see [https://developers.google.com/earth-engine/python_install-conda.html](https://developers.google.com/earth-engine/python_install-conda.html).

In [13]:
ee.Authenticate()

Enter verification code:  4/1AWtgzh7RLOFbR6Blq08h8mrrVZZlDpHbjt1mZ5ayX_DN19tQbHmypq4TMxE



Successfully saved authorization token.


In [28]:
!ls ../data

dhs_clusters.csv


In [14]:
ee.Initialize()  # initialize the Earth Engine API

## Constants

In [4]:
# ========== ADAPT THESE PARAMETERS ==========

# To export to Google Drive, uncomment the next 2 lines
# EXPORT = 'drive'
# BUCKET = None

# To export to Google Cloud Storage (GCS), uncomment the next 2 lines
# and set the bucket to the desired bucket name
EXPORT = 'gcs'
BUCKET = 'dhs-raw-images'

# # export location parameters
DHS_EXPORT_FOLDER = 'dhs_tfrecords_raw'
# LSMS_EXPORT_FOLDER = 'lsms_tfrecords_raw'

In [35]:
!pwd

/home/saori/code/saoriyosano/poverty_mapper/download


In [5]:
# ========== DO NOT MODIFY THESE ==========

# input data paths
DHS_CSV_PATH = '../data/dhs_clusters.csv'
# LSMS_CSV_PATH = '../data/lsms_clusters.csv'

# band names
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1']

# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
SCALE = 30                # export resolution: 30m/px
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 50        # set to a small number (<= 50) if Google Earth Engine reports memory errors

## Export Images

In [6]:
def export_images(
        df: pd.DataFrame,
        country: str,
        year: int,
        export_folder: str,
        chunk_size: Optional[int] = None,
        ) -> Dict[Tuple[Any], ee.batch.Task]:
    '''
    Args
    - df: pd.DataFrame, contains columns ['lat', 'lon', 'country', 'year']
    - country: str, together with `year` determines the survey to export
    - year: int, together with `country` determines the survey to export
    - export_folder: str, name of folder for export
    - chunk_size: int, optionally set a limit to the # of images exported per TFRecord file
        - set to a small number (<= 50) if Google Earth Engine reports memory errors

    Returns: dict, maps task name tuple (export_folder, country, year, chunk) to ee.batch.Task
    '''
    subset_df = df[(df['country'] == country) & (df['year'] == year)].reset_index(drop=True)
    if chunk_size is None:
        num_chunks = 1
    else:
        num_chunks = int(math.ceil(len(subset_df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)  # df.loc[] is inclusive
        fc = ee_utils.df_to_fc(subset_df.loc[chunk_slice, :])
        start_date, end_date = ee_utils.surveyyear_to_range(year)

        # create 3-year Landsat composite image
        roi = fc.geometry()
        imgcol = ee_utils.LandsatSR(roi, start_date=start_date, end_date=end_date).merged
        imgcol = imgcol.map(ee_utils.mask_qaclear).select(MS_BANDS)
        img = imgcol.median()

        # add nightlights, latitude, and longitude bands
        img = ee_utils.add_latlon(img)
        img = img.addBands(ee_utils.composite_nl(year))

        fname = f'{country}_{year}_{i:02d}'
        tasks[(country, year, i)] = ee_utils.get_array_patches(
            img=img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export=EXPORT,
            prefix=export_folder, fname=fname,
            bucket=BUCKET)
    return tasks

In [9]:
dhs_df = pd.read_csv(DHS_CSV_PATH, float_precision='high', index_col=False)
dhs_surveys = list(dhs_df.groupby(['country', 'year']).groups.keys())
tasks = {}

In [10]:
dhs_surveys[0:11]

[('angola', 2011),
 ('angola', 2015),
 ('benin', 2012),
 ('burkina_faso', 2010),
 ('burkina_faso', 2014),
 ('cameroon', 2011),
 ('cote_d_ivoire', 2012),
 ('democratic_republic_of_congo', 2013),
 ('ethiopia', 2010),
 ('ethiopia', 2016),
 ('ghana', 2014)]

In [ ]:
dhs_surveys

In [11]:
len(dhs_surveys)

43

In [15]:
for country, year in dhs_surveys[11:]:
    new_tasks = export_images(
        df=dhs_df, country=country, year=year,
        export_folder=DHS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)
    tasks.update(new_tasks)

Check on the status of each export task at [https://code.earthengine.google.com/](https://code.earthengine.google.com/), or run the following cell which checks every minute. Once all tasks have completed, download the DHS TFRecord files to `data/dhs_tfrecords_raw/` and LSMS TFRecord files to `data/lsms_tfrecords_raw/`.

In [ ]:
ee_utils.wait_on_tasks(tasks, poll_interval=60)

  0%|▎                                                                                  | 1/296 [00:01<05:08,  1.05s/it]

Task ('ghana', 2016, 0) finished in 0 min with state: COMPLETED


  1%|▌                                                                                  | 2/296 [00:02<05:12,  1.06s/it]

Task ('ghana', 2016, 1) finished in 0 min with state: COMPLETED


  1%|▊                                                                                  | 3/296 [00:03<05:11,  1.06s/it]

...

Task ('mozambique', 2011, 4) finished in 5 min with state: COMPLETED


 43%|███████████████████████████████████                                              | 128/296 [01:47<02:56,  1.05s/it]

Task ('mozambique', 2011, 5) finished in 5 min with state: COMPLETED


 44%|███████████████████████████████████▎                                             | 129/296 [01:48<02:55,  1.05s/it]

Task ('mozambique', 2011, 6) finished in 5 min with state: COMPLETED


In [25]:
# lsms_df = pd.read_csv(LSMS_CSV_PATH, float_precision='high', index_col=False)
# lsms_surveys = list(lsms_df.groupby(['country', 'year']).groups.keys())

# for country, year in lsms_surveys:
#     new_tasks = export_images(
#         df=lsms_df, country=country, year=year,
#         export_folder=LSMS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)
#     tasks.update(new_tasks)

FileNotFoundError: [Errno 2] No such file or directory: '../data/lsms_clusters.csv'